<a href="https://colab.research.google.com/github/pampamda/Chexpert_LLaVA-1.5-7B/blob/finetune/finetune_script/Unsloth_llava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tune

In [ ]:
from google.colab import drive
import os
from pathlib import Path

print("\nMounting Google Drive...")
drive.mount('/content/drive')


Mounting Google Drive...
Mounted at /content/drive


In [ ]:
DRIVE_DATA_PATH = "/content/drive/MyDrive/Group_Project/IS"
LOCAL_PATH = "/content/"
CSV_FILE = "data/processed_chexpert_reports_6000.csv"
MVP2000_JSON_FILE = "training_data_2000.json"
IMAGE_FOLDER = "data/filtered_chexpert_images_6000"

BASE_ADPATER_FOLDER = "llava_finetuned_mvp2000_v2"
MODEL_FOLDER = "llava_finetuned_mvp6000"
HF_FOLDER = "hf/llava_mvp6000"
MODEL_ID = "llava-hf/llava-1.5-7b-hf"

train_csv_path = os.path.join(DRIVE_DATA_PATH, CSV_FILE)
train_2000_json_path = os.path.join(DRIVE_DATA_PATH, MVP2000_JSON_FILE)
image_folder_path = os.path.join(DRIVE_DATA_PATH, IMAGE_FOLDER)
image_local_folder_path = "/content/filtered_chexpert_images_6000"

model_output_path = os.path.join(DRIVE_DATA_PATH, MODEL_FOLDER)
base_adapter_path = os.path.join(DRIVE_DATA_PATH, BASE_ADPATER_FOLDER)  # start with previoius model
hf_output_path = os.path.join(DRIVE_DATA_PATH, HF_FOLDER)

print("checkpoint folder: ", model_output_path)
print("hf folder: ", hf_output_path)

checkpoint folder:  /content/drive/MyDrive/Group_Project/IS/llava_finetuned_mvp6000
hf folder:  /content/drive/MyDrive/Group_Project/IS/hf/llava_mvp6000


In [ ]:
# prompt_template = f"<image>\nYou are an experienced radiologist. Based on the chest X-ray image and clinical context: {support_info}, generate a report with 3 sections. Prioritize accuracy—only describe what is clearly observed; if no relevant findings exist, state 'No evidence of...' or 'Unremarkable'.\n\n" \
#                   f"1. FINDINGS: Objectively describe all observed details, including:\n" \
#                   f"   - If present: Implants/devices (e.g., central lines, drains, stents, surgical clips)\n" \
#                   f"   - Abnormalities (e.g., edema, effusions, atelectasis) and normal findings, using standard radiological terminology.\n\n" \
#                   f"2. IMPRESSION: Concisely summarize key diagnostic conclusions based on FINDINGS, highlighting the most clinically significant abnormalities (e.g., 'Pulmonary vascular congestion without frank edema').\n\n" \
#                   f"3. SUMMARY: Note significance of findings. If no actionable issues, state 'No findings requiring immediate action'."

In [ ]:
!pip install -U transformers
!pip install peft
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 156.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-pze4mwnm/unsloth_96f514e17fd44732a0a170cbbe3ddd08
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-pze4mwnm/unsloth_96f514e17fd44732a0a170cbbe3ddd08
  Resolved https://github.com/unslothai/unsloth.git to commit a193011ba4a23610708e841adf910c84e341a25e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 44

In [ ]:
import torch
from unsloth import FastLanguageModel
from transformers import (
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, PeftModel

# Select model
print(f"\nUsing model: {MODEL_ID}")

max_seq_length = 2048 # Set max sequence length
dtype = torch.float16 # Set dtype

# load base model from llava
try:
    model, processor = FastLanguageModel.from_pretrained(
        model_name = MODEL_ID,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = True, # Use 4-bit quantization
        device_map="auto",
    )

    tokenizer = processor.tokenizer


    print("✅ Model loaded successfully with unsloth!")
    print("Model config vocab size:", model.config.vocab_size)
    print("Tokenizer vocab size:", tokenizer.vocab_size)
except Exception as e:
    print(f"❌ Model loading failed: {e}")
    import sys
    sys.exit(1)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!

Using model: llava-hf/llava-1.5-7b-hf
==((====))==  Unsloth 2025.10.3: Fast Clip patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.04G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

✅ Model loaded successfully with unsloth!
Model config vocab size: 32064
Tokenizer vocab size: 32000


In [ ]:
# load previous adapter, continue training
# skip if training from basic llava

model = PeftModel.from_pretrained(model, base_adapter_path, is_trainable=True)

model.print_trainable_parameters()

trainable params: 9,961,472 || all params: 7,073,388,544 || trainable%: 0.1408


In [ ]:
# Configure LoRA parameters with unsloth...
print("\nConfigure LoRA parameters with unsloth's LoraConfig...")

FastLoraConfig = None
try:
    # Attempt to import unsloth's LoraConfig
    from unsloth.models.lora import LoraConfig as FastLoraConfig
    print("Successfully imported LoraConfig from unsloth.lora.")
    using_unsloth_config = True
except ImportError:
    print("Could not import LoraConfig from unsloth.lora.")
    # Fallback to peft.LoraConfig
    try:
        from peft import LoraConfig as FastLoraConfig
        print("Falling back to using peft.LoraConfig.")
        using_unsloth_config = False
    except ImportError:
        print("FATAL ERROR: Could not import LoraConfig from either unsloth.lora or peft.")
        FastLoraConfig = None # Ensure FastLoraConfig is None on fatal failure
        using_unsloth_config = False # Indicate failure to use any config


if FastLoraConfig is not None:
    lora_config = FastLoraConfig(
        r=16,  # LoRA rank
        lora_alpha=32,  # LoRA alpha
        target_modules=["q_proj", "v_proj"], # only modify the attention layers
        lora_dropout=0.05,  # Dropout rate
        bias="none",  # don't train bias
        task_type="CAUSAL_LM"  # task type: causal language model
    )

    print("\nLoRA configuration details:")
    print(f"  - Using unsloth's config: {using_unsloth_config}")
    print(f"  - Rank (r): {lora_config.r}")
    print(f"  - Alpha: {lora_config.lora_alpha}")
    print(f"  - Target modules: {len(lora_config.target_modules)} layers")
    print(f"  - Dropout: {lora_config.lora_dropout}")
    print("\n✅ LoRA configuration prepared.")
else:
    print("\n❌ Failed to prepare LoRA configuration due to import errors.")



Configure LoRA parameters with unsloth's LoraConfig...
Could not import LoraConfig from unsloth.lora.
Falling back to using peft.LoraConfig.

LoRA configuration details:
  - Using unsloth's config: False
  - Rank (r): 16
  - Alpha: 32
  - Target modules: 2 layers
  - Dropout: 0.05

✅ LoRA configuration prepared.


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import os

class ChestXrayDataset(Dataset):
    def __init__(self, data_list, processor, format, image_root="", max_length=1024):
        self.data = data_list
        self.processor = processor
        self.format = format
        self.image_root = image_root
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        prompt_text = "" # Initialize prompt_text
        full_text = "" # Initialize full_text

        if self.format == "json":
          item = self.data[idx]
          # get image path (supports either "image_path" or "image")
          img_path = item.get("image_path") or item.get("image")
          if self.image_root and not os.path.isabs(img_path):
              img_path = os.path.join(self.image_root, img_path)

          # load image
          image = Image.open(img_path).convert("RGB")

          # extract question and answer
          conversations = item['conversations']
          question = conversations[0]['value']
          answer = conversations[1]['value']

          # If question already contains "<image>", don't add another one.
          if "<image>" in question:
              prompt_text = f"User: {question}\nAssistant:"
          else:
              prompt_text = f"User: <image>\n{question}\nAssistant:"

          full_text = prompt_text + " " + answer
        elif self.format == "csv":
          row = self.data.iloc[idx]

          img_name = row["image_name"]
          img_path = os.path.join(self.image_root, img_name) # Corrected image path construction
          image = Image.open(img_path).convert("RGB")

          # prompt构造
          support_info = str(row.get("support_information", "")).strip()
          combined_content = str(row.get("combined_content", "")).strip()

          question = (
              f"<image>\nYou are an experienced radiologist. Based on the chest X-ray image and clinical context: {support_info}, generate a report with 3 sections. Prioritize accuracy—only describe what is clearly observed; if no relevant findings exist, state 'No evidence of...' or 'Unremarkable'.\n\n" \
                  f"1. FINDINGS: Objectively describe all observed details, including:\n" \
                  f"   - If present: Implants/devices (e.g., central lines, drains, stents, surgical clips)\n" \
                  f"   - Abnormalities (e.g., edema, effusions, atelectasis) and normal findings, using standard radiological terminology.\n\n" \
                  f"2. IMPRESSION: Concisely summarize key diagnostic conclusions based on FINDINGS, highlighting the most clinically significant abnormalities (e.g., 'Pulmonary vascular congestion without frank edema').\n\n" \
                  f"3. SUMMARY: Note significance of findings. If no actionable issues, state 'No findings requiring immediate action'."
          )
          answer = combined_content

          prompt_text = f"User: {question}\nAssistant:"

          # 构造 full_text
          full_text = prompt_text + " " + answer


        # Use consistent tokenization settings for both prompt and full sequence
        prompt_tokens = self.processor(
            text=prompt_text,
            images=image,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=self.max_length
        )
        full_tokens = self.processor(
            text=full_text,
            images=image,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=self.max_length
        )

        # get tensors (remove batch dim and clone)
        prompt_input_ids = prompt_tokens["input_ids"][0].clone() # Clone here
        input_ids = full_tokens["input_ids"][0].clone() # Clone here
        attention_mask = full_tokens["attention_mask"][0].clone() # Clone here

        pixel_values = full_tokens.get("pixel_values")
        if pixel_values is not None:
            pixel_values = pixel_values[0].clone() # Clone here

        # compute prompt length robustly
        pad_token_id = self.processor.tokenizer.pad_token_id
        if pad_token_id is None:
            pad_token_id = self.processor.tokenizer.eos_token_id

        prompt_len = (prompt_input_ids != pad_token_id).sum().item()

        # create labels by masking prompt portion to -100
        labels = input_ids.clone() # Clone here
        if prompt_len > 0:
            labels[:prompt_len] = -100

        out = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }
        if pixel_values is not None:
            out["pixel_values"] = pixel_values

        return out

In [ ]:
# copy images to local, improve training speed
# 10min?

!cp -r /content/drive/MyDrive/Group_Project/IS/data/filtered_chexpert_images_6000 /content/filtered_chexpert_images_6000

In [ ]:
import json
import pandas as pd

output_json = train_2000_json_path

# load mvp2000 data
# with open(output_json, 'r', encoding='utf-8') as f:
#     training_data = json.load(f)
# train_dataset = ChestXrayDataset(training_data, processor, "json", "")

# load mvp6000 data
training_data = pd.read_csv(train_csv_path)
train_dataset = ChestXrayDataset(training_data, processor, "csv", image_local_folder_path)  # load data from local cache

print("✅ Dataset loaded successfully.")

✅ Dataset loaded successfully.


In [ ]:
if getattr(model, "peft_config", None) is None:
    print("\nApplying LoRA configuration to the model...")
    model = get_peft_model(model, lora_config)
    print("✅ LoRA applied successfully using get_peft_model!")
else:
    print("\nModel is already a PEFT model. Skipping get_peft_model.")

print("\nParameter statistics after applying LoRA:")
model.print_trainable_parameters()

OUTPUT_DIR = model_output_path

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3, # Keep 1 epoch for MVP
    per_device_train_batch_size=8,  # 单步batch从1→4（根据显存调整，24GB卡可试2-4）
    gradient_accumulation_steps=1,  # 累积步数从4→2，保持有效batch=4（2×2=4）不变
    gradient_checkpointing=False,  # 关闭梯度检查点
    learning_rate=2e-5,
    warmup_steps=100,
    # fp16=True,
    logging_steps=5,
    save_strategy="epoch",
    optim="paged_adamw_32bit",
    load_best_model_at_end=False,
    report_to="none",
    remove_unused_columns=False,

    # improve speed
    bf16=True,  # 强制启用BF16（A100原生支持，比FP16快30%）
    fp16=False,  # 关闭FP16
    torch_compile=True,
    torch_compile_backend="inductor",  # 用inductor后端优化A100的Tensor Core
)

print("\nTraining configuration:")
print(f"  - Training epochs: {training_args.num_train_epochs}")
print(f"  - Actual batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  - Learning rate: {training_args.learning_rate}")
# Calculate total steps
total_steps = len(train_dataset) * training_args.num_train_epochs // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)
print(f"  - Total training steps: ~{total_steps}")
print(f"  - Output directory: {OUTPUT_DIR}")

print("\nInstantiating Trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=processor.tokenizer, # Use tokenizer from the unsloth processor
)

print("✅ Trainer created successfully with PEFT model and unsloth processor/tokenizer.")



Model is already a PEFT model. Skipping get_peft_model.

Parameter statistics after applying LoRA:
trainable params: 9,961,472 || all params: 7,073,388,544 || trainable%: 0.1408

Training configuration:
  - Training epochs: 3
  - Actual batch size: 8
  - Learning rate: 2e-05
  - Total training steps: ~2513
  - Output directory: /content/drive/MyDrive/Group_Project/IS/llava_finetuned_mvp6000

Instantiating Trainer...
✅ Trainer created successfully with PEFT model and unsloth processor/tokenizer.


/tmp/ipython-input-349389574.py:46: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
print("\n" + "🚀" * 30)
print("Starting training!")
print("🚀" * 30)

print("\nEstimated training time (with unsloth acceleration):")
if 'total_steps' not in locals():
     print("Warning: total_steps variable not found. Recalculating...")
     # Recalculate total steps based on current training_args and train_dataset
     try:
         total_steps = len(train_dataset) * training_args.num_train_epochs // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)
         print(f"  - Total training steps: ~{total_steps}")
     except Exception as e:
         print(f"Could not calculate total steps: {e}")
         total_steps = "Unknown" # Set to unknown if calculation fails
else:
    print(f"  - Total training steps: {total_steps}")


print("  - Training speedup expected compared to previous runs.")
print("\nStarting training, please keep Colab tab open...\n")

try:
    train_result = trainer.train()
    print("\n" + "🎉" * 30)
    print("Training complete!")
    print("🎉" * 30)
    print(f"\nTraining statistics:")
    print(f"  - Total steps: {train_result.global_step}")
    print(f"  - Training loss: {train_result.training_loss:.4f}")

except Exception as e:
    print(f"\n❌ Training failed: {e}")
    print("\nPossible reasons:")
    print("1. Insufficient VRAM → reduce batch_size or r")
    print("2. Data issues → check images and reports (check dataset __getitem__ output)")
    print("3. Interrupted → re-run")
    import traceback
    traceback.print_exc()
    import sys
    sys.exit(1)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.



🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
Starting training!
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀

Estimated training time (with unsloth acceleration):
  - Total training steps: 2513
  - Training speedup expected compared to previous runs.

Starting training, please keep Colab tab open...



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,703 | Num Epochs = 3 | Total steps = 2,514
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 9,961,472 of 7,073,388,544 (0.14% trained)


Unsloth: Enabled auto compiling


AUTOTUNE convolution(8x3x336x336, 1024x3x14x14)
strides: [3*s100**2*s89, s100**2, s100, 1], [588, 196, 14, 1]
dtypes: torch.bfloat16, torch.bfloat16
  triton_convolution2d_6 0.4004 ms 100.0% ALLOW_TF32=True, BLOCK_K=16, BLOCK_M=256, BLOCK_N=64, GROUPS=1, KERNEL_H=14, KERNEL_W=14, PADDING_H=0, PADDING_W=0, STRIDE_H=14, STRIDE_W=14, UNROLL=False, num_stages=2, num_warps=8
  triton_convolution2d_1 0.4454 ms 89.9% ALLOW_TF32=True, BLOCK_K=16, BLOCK_M=256, BLOCK_N=64, GROUPS=1, KERNEL_H=14, KERNEL_W=14, PADDING_H=0, PADDING_W=0, STRIDE_H=14, STRIDE_W=14, UNROLL=False, num_stages=2, num_warps=4
  convolution 0.5284 ms 75.8% 
  triton_convolution2d_3 0.5325 ms 75.2% ALLOW_TF32=True, BLOCK_K=16, BLOCK_M=128, BLOCK_N=128, GROUPS=1, KERNEL_H=14, KERNEL_W=14, PADDING_H=0, PADDING_W=0, STRIDE_H=14, STRIDE_W=14, UNROLL=False, num_stages=2, num_warps=8
  triton_convolution2d_4 0.6062 ms 66.0% ALLOW_TF32=True, BLOCK_K=16, BLOCK_M=64, BLOCK_N=64, GROUPS=1, KERNEL_H=14, KERNEL_W=14, PADDING_H=0, PADDIN

Step,Training Loss
5,1.880000
10,1.870900
15,1.922700
20,1.818600
25,1.799400
30,1.852500
35,1.811500
40,1.805500
45,1.803000
50,1.780400


Unsloth: Will smartly offload gradients to save VRAM!

🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉
Training complete!
🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

Training statistics:
  - Total steps: 2514
  - Training loss: 0.9521


In [ ]:
trainer.save_model(OUTPUT_DIR)
processor.save_pretrained(OUTPUT_DIR)

print(f"✅ Model saved to: {OUTPUT_DIR}")

✅ Model saved to: /content/drive/MyDrive/Group_Project/IS/llava_finetuned_mvp6000


# Issue: overfitting

decoding collapse, retrain the mvp6000_v2